Load the data and the libraries

In [26]:
import pandas as pd
import json

In [6]:
data = pd.read_csv('../dataset/RecipeNLG_dataset.csv')
data.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


Just checking whether there are any difference in the data between the two data sources

In [7]:
data['source'].value_counts()

source
Gathered     1643098
Recipes1M     588044
Name: count, dtype: int64

In [21]:
data[data['source'] == 'Gathered'].head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [15]:
data[data['source'] == 'Recipes1M'].head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
1643098,1643098,Worlds Best Mac and Cheese,"[""6 ounces penne"", ""2 cups Beechers Flagship C...","[""Preheat the oven to 350 F. Butter or oil an ...",www.epicurious.com/recipes/food/views/-world-s...,Recipes1M,"[""penne"", ""Beechers Flagship Cheese Sauce"", ""C..."
1643099,1643099,Dilly Macaroni Salad Recipe,"[""1 c. elbow macaroni"", ""1 c. cubed American c...","[""Cook macaroni according to package direction...",cookeatshare.com/recipes/dilly-macaroni-salad-...,Recipes1M,"[""elbow macaroni"", ""American cheese"", ""celery""..."
1643100,1643100,Gazpacho,"[""8 tomatoes, quartered"", ""Kosher salt"", ""1 re...","[""Add the tomatoes to a food processor with a ...",www.foodnetwork.com/recipes/gazpacho1.html,Recipes1M,"[""tomatoes"", ""Kosher salt"", ""red onion"", ""gree..."
1643101,1643101,Kombu Tea Grilled Chicken Thigh,"[""2 Chicken thighs"", ""2 tsp Kombu tea"", ""1 Whi...","[""Pierce the skin of the chicken with a fork o...",cookpad.com/us/recipes/150100-kombu-tea-grille...,Recipes1M,"[""Chicken"", ""White pepper""]"
1643102,1643102,Zucchini Nut Bread,"[""2 cups flour"", ""1 tablespoon cinnamon"", ""2 t...","[""Sift dry ingredients."", ""beat eggs untill fr...",www.food.com/recipe/zucchini-nut-bread-329682,Recipes1M,"[""flour"", ""cinnamon"", ""baking soda"", ""salt"", ""..."


There are cases where the NER and actual ingredient entities are not matching!
So we can't use NER here, maybe we have to extract it from ingredients...

In [24]:
data.iloc[1643101]['ingredients']

'["2 Chicken thighs", "2 tsp Kombu tea", "1 White pepper"]'

In [23]:
data.iloc[1643101]['NER']

'["Chicken", "White pepper"]'

## Things we can extract from the dataset!

- Title of the recipe: Title
- ingredient: ingredients
- link: no need for FT, it might hallucinate
- directions: GOLD
- NER: GOLD (for getting specific elements in the recipe)

Let's convert this to a json format



In [28]:
data_json = data[['title', 'ingredients', 'directions']].to_json(orient='records', force_ascii=False, indent=2)


Save the data to json file

In [31]:
with open("recipes.json", "w") as f:
    f.write(data_json)

In [ ]:
# process the json file
with open("recipes.json", "r") as f:
    recipes = json.load(f)

recipes[:5]